In [27]:
import torch 
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [28]:
batch_size=64

In [29]:
transform=transforms.Compose([
    transforms.ToTensor(),#把0-255转换成0-1,把维度转换成C*W*H
    transforms.Normalize((0.1307,),(0.3081,))])#均值和标准差,归一化,

In [30]:
train_dataset=datasets.MNIST(root='../dataset/mnist/',
                             train=True,
                             download=True,
                             transform=transform)

In [31]:
train_loader=DataLoader(train_dataset,
                        shuffle=True,
                        batch_size=batch_size)

In [32]:
test_dataset=datasets.MNIST(root='../dataset/mnist/',
                           train=False,
                           download=True,
                           transform=transform)

In [33]:
test_loader=DataLoader(test_dataset,
                      shuffle=False,
                      batch_size=batch_size)

In [34]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1=torch.nn.Linear(784,512)
        self.l2=torch.nn.Linear(512,256)
        self.l3=torch.nn.Linear(256,128)
        self.l4=torch.nn.Linear(128,64)
        self.l5=torch.nn.Linear(64,10)
    def forward(self,x):
        x=x.view(-1,784)
        x=F.relu(self.l1(x))
        x=F.relu(self.l2(x))
        x=F.relu(self.l3(x))
        x=F.relu(self.l4(x))
        return self.l5(x)
model=Net()

In [35]:
criterion=torch.nn.CrossEntropyLoss()

In [36]:
optimizer=optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

In [37]:
def train(epoch):
    running_loss=0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs, target = data
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0        

In [38]:
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%'%(100*correct/total))

In [39]:
for epoch in range(10):
    train(epoch)
    test()

[1,   300] loss: 2.193
[1,   600] loss: 0.819
[1,   900] loss: 0.422
Accuracy on test set: 90 %
[2,   300] loss: 0.317
[2,   600] loss: 0.266
[2,   900] loss: 0.223
Accuracy on test set: 93 %
[3,   300] loss: 0.191
[3,   600] loss: 0.165
[3,   900] loss: 0.151
Accuracy on test set: 96 %
[4,   300] loss: 0.132
[4,   600] loss: 0.122
[4,   900] loss: 0.113
Accuracy on test set: 96 %
[5,   300] loss: 0.096
[5,   600] loss: 0.096
[5,   900] loss: 0.090
Accuracy on test set: 96 %
[6,   300] loss: 0.077
[6,   600] loss: 0.077
[6,   900] loss: 0.076
Accuracy on test set: 97 %
[7,   300] loss: 0.063
[7,   600] loss: 0.061
[7,   900] loss: 0.059
Accuracy on test set: 97 %
[8,   300] loss: 0.047
[8,   600] loss: 0.048
[8,   900] loss: 0.051
Accuracy on test set: 97 %
[9,   300] loss: 0.038
[9,   600] loss: 0.044
[9,   900] loss: 0.039
Accuracy on test set: 97 %
[10,   300] loss: 0.031
[10,   600] loss: 0.032
[10,   900] loss: 0.036
Accuracy on test set: 97 %


权重不够多,局部信息的利用不够好,特征提取欸有,例如傅里叶变换,高级特征

高抽象级别的特征,现在比较原始